In [1]:
import pandas as pd
from gprofiler import GProfiler
gp = GProfiler(return_dataframe=True)

In [2]:
delta_correlation_df = pd.read_csv('delta_correlation_df.csv')
delta_correlation_df

,Gene,Delta_Correlation,P_Value,FDR,Cancer
0,A1BG,-0.268533,5.703182e-02,1.320375e-01,CCRCC
1,A1CF,0.192038,1.063340e-04,6.401858e-04,CCRCC
2,A2M,-0.191619,1.277644e-01,2.439276e-01,CCRCC
3,AAAS,0.019654,8.963138e-01,9.409267e-01,CCRCC
4,AACS,-0.169937,6.007042e-02,1.375402e-01,CCRCC
...,...,...,...,...,...
50303,ZXDC,0.073854,6.395306e-01,6.962162e-01,LUAD
50304,ZYG11B,0.734569,1.583743e-10,1.504331e-09,LUAD
50305,ZYX,0.440861,2.253307e-06,9.499585e-06,LUAD
50306,ZZEF1,0.422001,3.731185e-11,3.889196e-10,LUAD


In [3]:
all_pathways = []
positive_pathways = []
negative_pathways = []
for cancer in pd.unique(delta_correlation_df.Cancer):
    cancer_df = delta_correlation_df[delta_correlation_df.Cancer == cancer]
    background_genes  = list(pd.unique(cancer_df.Gene))
    cancer_df = cancer_df[cancer_df.FDR > 0.05]
    positive_cancer_df = cancer_df[cancer_df.Delta_Correlation > 0]
    negative_cancer_df = cancer_df[cancer_df.Delta_Correlation < 0]
    cancer_df['Delta_Correlation'] = abs(cancer_df['Delta_Correlation'])
    cancer_df = cancer_df.sort_values('Delta_Correlation', ascending = False)
    positive_cancer_df = positive_cancer_df.sort_values('Delta_Correlation', ascending = False)
    negative_cancer_df = negative_cancer_df.sort_values('Delta_Correlation')
    all_genes = list(cancer_df.Gene)
    positive_genes = list(positive_cancer_df.Gene)
    negative_genes = list(negative_cancer_df.Gene)
    all_cancer_pathways = gp.profile(organism='hsapiens', query = all_genes, no_iea=True,
                       ordered=True, no_evidences=False, background= background_genes)
    all_cancer_pathways['Cancer'] = [cancer] * len(all_cancer_pathways)
    positive_cancer_pathways = gp.profile(organism='hsapiens', query = positive_genes, no_iea=True,
                       ordered=True, no_evidences=False, background= background_genes)
    positive_cancer_pathways['Cancer'] = [cancer] * len(positive_cancer_pathways)
    negative_cancer_pathways = gp.profile(organism='hsapiens', query = negative_genes, no_iea=True,
                       ordered=True, no_evidences=False, background= background_genes)
    negative_cancer_pathways['Cancer'] = [cancer] * len(negative_cancer_pathways)
    all_pathways.append(all_cancer_pathways)
    positive_pathways.append(positive_cancer_pathways)
    negative_pathways.append(negative_cancer_pathways)
all_pathways = pd.concat(all_pathways)    
all_pathways.to_csv('Individual_cancer_pathway_enrichment.csv', index = False)
positive_pathways = pd.concat(positive_pathways)
positive_pathways.to_csv('Positive_delta_corr_individual_cancer_pathways.csv', index = False)
negative_pathways = pd.concat(negative_pathways)
negative_pathways.to_csv('Negative_delta_corr_individual_cancer_pathways', index = False)

In [4]:
all_pathways

,source,native,name,p_value,significant,description,term_size,query_size,intersection_size,effective_domain_size,precision,recall,query,parents,intersections,evidences,Cancer
0,CORUM,CORUM:351,Spliceosome,0.002455,True,Spliceosome,135,6424,112,9916,0.017435,0.829630,query_1,[CORUM:0000000],"[THOC5, CDC40, SF3A3, SRSF9, SRSF2, SRRM2, SMU...","[[CORUM], [CORUM], [CORUM], [CORUM], [CORUM], ...",CCRCC
1,CORUM,CORUM:1201,TRF1-TIN2 complex,0.002743,True,TRF1-TIN2 complex,4,1020,4,9916,0.003922,1.000000,query_1,[CORUM:0000000],"[POT1, TINF2, TERF1, ACD]","[[CORUM], [CORUM], [CORUM], [CORUM]]",CCRCC
2,CORUM,CORUM:610,MSL complex,0.002988,True,MSL complex,4,1042,4,9916,0.003839,1.000000,query_1,[CORUM:0000000],"[MSL2, KANSL1, MSL3, KAT8]","[[CORUM], [CORUM], [CORUM], [CORUM]]",CCRCC
3,CORUM,CORUM:3061,"RNA polymerase II complex (CBP, PCAF, RPB1, BA...",0.007814,True,"RNA polymerase II complex (CBP, PCAF, RPB1, BA...",5,320,3,9916,0.009375,0.600000,query_1,[CORUM:0000000],"[KAT2B, CDK8, SMARCB1]","[[CORUM], [CORUM], [CORUM]]",CCRCC
4,CORUM,CORUM:1544,PPP2CA-PPP2R1A-PPP2R3A complex,0.009237,True,PPP2CA-PPP2R1A-PPP2R3A complex,3,716,3,9916,0.004190,1.000000,query_1,[CORUM:0000000],"[PPP2R1A, PPP2R3A, PPP2CA]","[[CORUM], [CORUM], [CORUM]]",CCRCC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44,HP,HP:0030270,Elevated red cell adenosine deaminase level,0.018077,True,Increase in the level of adenosine deaminase (...,20,3583,18,9998,0.005024,0.900000,query_1,[HP:0030272],"[RPL26, RPS26, RPS28, RPS19, RPS27, RPS24, RPL...","[[HP], [HP], [HP], [HP], [HP], [HP], [HP], [HP...",LUAD
45,HP,HP:0031640,Abnormal radial artery morphology,0.018077,True,Any structural anomaly of the radial artery.,20,3583,18,9998,0.005024,0.900000,query_1,[HP:0011004],"[RPL26, RPS26, RPS28, RPS19, RPS27, RPS24, RPL...","[[HP], [HP], [HP], [HP], [HP], [HP], [HP], [HP...",LUAD
46,HP,HP:0003048,Radial head subluxation,0.026460,True,Partial dislocation of the head of the radius.,3,268,3,9998,0.011194,1.000000,query_1,[HP:0032153],"[XYLT1, B3GALT6, GLI3]","[[HP], [HP], [HP]]",LUAD
47,CORUM,CORUM:230,Mediator complex,0.036078,True,Mediator complex,27,3022,19,9998,0.006287,0.703704,query_1,[CORUM:0000000],"[MED22, MED18, MED21, MED25, MED28, MED9, MED3...","[[CORUM], [CORUM], [CORUM], [CORUM], [CORUM], ...",LUAD


In [5]:
positive_pathways

,source,native,name,p_value,significant,description,term_size,query_size,intersection_size,effective_domain_size,precision,recall,query,parents,intersections,evidences,Cancer
0,CORUM,CORUM:1544,PPP2CA-PPP2R1A-PPP2R3A complex,0.009439,True,PPP2CA-PPP2R1A-PPP2R3A complex,3,523,3,9916,0.005736,1.000000,query_1,[CORUM:0000000],"[PPP2R1A, PPP2R3A, PPP2CA]","[[CORUM], [CORUM], [CORUM]]",CCRCC
1,CORUM,CORUM:3060,"RNA polymerase II complex (RPB1, RAP74, CDK8, ...",0.017224,True,"RNA polymerase II complex (RPB1, RAP74, CDK8, ...",6,2517,6,9916,0.002384,1.000000,query_1,[CORUM:0000000],"[CDK8, SMARCB1, GTF2F1, SMARCA2, MED21, CCNC]","[[CORUM], [CORUM], [CORUM], [CORUM], [CORUM], ...",CCRCC
2,CORUM,CORUM:6816,ULK1-ATG13-ATG14-PIK3C3 complex,0.020421,True,ULK1-ATG13-ATG14-PIK3C3 complex,3,431,3,9916,0.006961,1.000000,query_1,[CORUM:0000000],"[ULK1, ATG14, PIK3C3]","[[CORUM], [CORUM], [CORUM]]",CCRCC
3,CORUM,CORUM:1307,Multiprotein complex (mRNA turnover),0.043333,True,Multiprotein complex (mRNA turnover),4,1093,4,9916,0.003660,1.000000,query_1,[CORUM:0000000],"[HNRNPD, PAIP1, SYNCRIP, PABPC1]","[[CORUM], [CORUM], [CORUM], [CORUM]]",CCRCC
4,CORUM,CORUM:1308,PABPC1-HSPA8-HNRPD-EIF4G1 complex,0.043380,True,PABPC1-HSPA8-HNRPD-EIF4G1 complex,4,1597,4,9916,0.002505,1.000000,query_1,[CORUM:0000000],"[EIF4G1, HNRNPD, PABPC1, HSPA8]","[[CORUM], [CORUM], [CORUM], [CORUM]]",CCRCC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35,GO:CC,GO:0044391,ribosomal subunit,0.003946,True,"""Either of the two subunits of a ribosome: the...",172,2420,77,9998,0.031818,0.447674,query_1,"[GO:0005840, GO:1990904]","[MRPL34, RPL26, MRPL23, RPS18, RPL18A, MRPL42,...","[[IDA, IBA], [HDA, IBA, TAS], [IDA, IBA, TAS],...",LUAD
36,GO:CC,GO:0022625,cytosolic large ribosomal subunit,0.007558,True,"""The large subunit of a ribosome located in th...",49,2641,33,9998,0.012495,0.673469,query_1,"[GO:0015934, GO:0022626]","[RPL26, RPL18A, RPL7, RPL26L1, RPL4, RPL10, RP...","[[HDA, IBA, TAS], [IDA, HDA, IBA, TAS], [HDA, ...",LUAD
37,GO:BP,GO:0019080,viral gene expression,0.015516,True,"""A process by which a viral gene is converted ...",188,2653,98,9998,0.036939,0.521277,query_1,[GO:0016032],"[ZNF639, TRIM11, RPL26, RPS18, TUSC3, RPL18A, ...","[[IMP], [IDA], [TAS], [TAS], [TAS], [TAS], [IM...",LUAD
38,HP,HP:0003048,Radial head subluxation,0.019001,True,Partial dislocation of the head of the radius.,3,226,3,9998,0.013274,1.000000,query_1,[HP:0032153],"[XYLT1, B3GALT6, GLI3]","[[HP], [HP], [HP]]",LUAD


In [6]:
negative_pathways

,source,native,name,p_value,significant,description,term_size,query_size,intersection_size,effective_domain_size,precision,recall,query,parents,intersections,evidences,Cancer
0,CORUM,CORUM:320,"55S ribosome, mitochondrial",1.849093e-08,True,"55S ribosome, mitochondrial",77,2094,42,9916,0.020057,0.545455,query_1,[CORUM:0000000],"[MRPS18B, MRPL23, MRPL24, MRPS22, MRPL40, MRPS...","[[CORUM], [CORUM], [CORUM], [CORUM], [CORUM], ...",CCRCC
1,REAC,REAC:R-HSA-5368286,Mitochondrial translation initiation,3.074417e-08,True,Mitochondrial translation initiation,86,2094,49,9916,0.023400,0.569767,query_1,[REAC:R-HSA-5368287],"[MRPS18B, MRPL23, MRPL24, MRPS22, MRPL40, MRPS...","[[REAC], [REAC], [REAC], [REAC], [REAC], [REAC...",CCRCC
2,REAC,REAC:R-HSA-5368287,Mitochondrial translation,4.260379e-08,True,Mitochondrial translation,91,2094,51,9916,0.024355,0.560440,query_1,[REAC:R-HSA-72766],"[MRPS18B, MRPL23, MRPL24, MRPS22, MRPL40, MRPS...","[[REAC], [REAC], [REAC], [REAC], [REAC], [REAC...",CCRCC
3,REAC,REAC:R-HSA-5419276,Mitochondrial translation termination,1.486876e-07,True,Mitochondrial translation termination,86,2094,48,9916,0.022923,0.558140,query_1,[REAC:R-HSA-5368287],"[MRPS18B, MRPL23, MRPL24, MRPS22, MRPL40, MRPS...","[[REAC], [REAC], [REAC], [REAC], [REAC], [REAC...",CCRCC
4,GO:BP,GO:0070126,mitochondrial translational termination,2.607585e-07,True,"""The process resulting in the release of a pol...",87,2094,48,9916,0.022923,0.551724,query_1,"[GO:0006415, GO:0032543]","[MRPS18B, MRPL23, MRPL24, MRPS22, MRPL40, MRPS...","[[TAS], [TAS], [TAS], [TAS], [TAS], [TAS], [TA...",CCRCC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34,WP,WP:WP558,Complement and Coagulation Cascades,3.597328e-02,True,Complement and Coagulation Cascades,53,911,15,9998,0.016465,0.283019,query_1,[WP:000000],"[F8, SERPIND1, SERPINC1, CPB2, CFH, MASP2, KLK...","[[WP], [WP], [WP], [WP], [WP], [WP], [WP], [WP...",LUAD
35,CORUM,CORUM:2903,Respiratory chain complex I (intermediate V/38...,3.688544e-02,True,Respiratory chain complex I (intermediate V/38...,5,292,3,9998,0.010274,0.600000,query_1,[CORUM:0000000],"[NDUFS3, MT-ND1, NDUFS2]","[[CORUM], [CORUM], [CORUM]]",LUAD
36,WP,WP:WP4324,Mitochondrial complex I assembly model OXPHOS ...,4.360182e-02,True,Mitochondrial complex I assembly model OXPHOS ...,47,298,8,9998,0.026846,0.170213,query_1,[WP:000000],"[NDUFV1, NDUFS3, MT-ND1, NDUFC1, NDUFB5, NDUFA...","[[WP], [WP], [WP], [WP], [WP], [WP], [WP], [WP]]",LUAD
37,HP,HP:0010451,Aplasia/Hypoplasia of the spleen,4.400642e-02,True,Absence or underdevelopment of the spleen.,52,684,17,9998,0.024854,0.326923,query_1,[HP:0025408],"[LRRC6, MNS1, CFAP221, DNAAF3, GAS2L2, CCDC40,...","[[HP], [HP], [HP], [HP], [HP], [HP], [HP], [HP...",LUAD
